In [201]:
# import
import os
import glob
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm

import time
import torch
import tensorly as tl
import nibabel as nib
import matplotlib.pyplot as plt
from tensorly.metrics.regression import RMSE
from tensorly.decomposition import non_negative_tucker, non_negative_tucker_hals,tucker

path = r'E:\Thanh\Projects\QG-Alzheirmer\tensorFeatureExtraction\data\\'
pair = ['AD','NC']
rank = [30,30,25]
rank_str = '_'.join([str(elem) for elem in rank])
combine_paths = ([glob.glob( path +fr'{x}_tucker_{rank_str}//*.p') for x in pair])
combine_paths = combine_paths[0] + combine_paths[1]




In [204]:
combine_paths[0]

'E:\\Thanh\\Projects\\QG-Alzheirmer\\tensorFeatureExtraction\\data\\\\AD_tucker_30_30_25\\002_S_0619_tucker_30,30,25.p'

In [ ]:
Nsubject_core_tensor

In [202]:
core_tensor = []
for x in range(3):
    factor_dict[f"factor{x}"] = []
for path in tqdm(combine_paths):
    with open(path, 'rb') as f:
        data = pickle.load(f)
        core =  np.array(data[0])
        factors = data[1]
        core_tensor.append(core)
        [factor_dict[f"factor{x}"].append(factors[x]) for x in range(3)]

Nsubject_core_tensor = np.array(core_tensor)
Nsubject_factor0_tensor = np.array(factor_dict["factor0"])
Nsubject_factor1_tensor = np.array(factor_dict["factor1"])
Nsubject_factor2_tensor = np.array(factor_dict["factor2"])

100%|███████████████████████████████████████████████| 791/791 [00:01<00:00, 439.11it/s]


In [198]:
Nsubject_core_tensor.shape,Nsubject_factor2_tensor.shape,Nsubject_factor1_tensor.shape,Nsubject_factor0_tensor.shape

((0,), (0,), (0,), (0,))

In [ ]:
core_factor_T = [torch.from_numpy(x.T).to('cuda') for x in core_factor[1:]]
factor0_factor_T  = [torch.from_numpy(x.T).to('cuda') for x in factor0_factor[1:]]
factor1_factor_T  = [torch.from_numpy(x.T).to('cuda') for x in factor1_factor[1:]]
factor2_factor_T  = [torch.from_numpy(x.T).to('cuda') for x in factor2_factor[1:]]
with open('factor_all_subject.p', 'wb') as f:
    pickle.dump((core_factor_T,factor0_factor_T,factor1_factor_T,factor2_factor_T), f)
    

In [192]:

for path in tqdm(combine_paths):
    # print(path)
    with open(path, 'rb') as f:
        data = pickle.load(f)
        core =  np.array(data[0])
        factors = data[1]
        
        extract_core = tl.tenalg.multi_mode_dot(torch.from_numpy(core).to('cuda'),core_factor_T)
        extract_factor0 = tl.tenalg.multi_mode_dot(torch.from_numpy(factors[0]).to('cuda'),factor0_factor_T)
        extract_factor1 = tl.tenalg.multi_mode_dot(torch.from_numpy(factors[1]).to('cuda'),factor1_factor_T)
        extract_factor2 = tl.tenalg.multi_mode_dot(torch.from_numpy(factors[2]).to('cuda'),factor2_factor_T)
        
        saveDir = os.path.join(os.path.dirname(path), 'feature')
        if not os.path.exists(saveDir):
            os.mkdir(saveDir)
        with open(os.path.join(saveDir, os.path.basename(path)),'wb') as f:
            pickle.dump((extract_core.cpu().numpy(),extract_factor0.cpu().numpy(),extract_factor1.cpu().numpy(),extract_factor2.cpu().numpy()),f)
        

100%|███████████████████████████████████████████████| 791/791 [00:04<00:00, 165.37it/s]
